In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

In [ ]:
class ImageStitcher:
    
    def __init__(self, imgs):
        self.imgs=imgs


    def __denormalize_homographies(self, H, T_norm):
        H_denorm = [ np.linalg.inv(T_norm) @ h @ T_norm  for h in H]
        return H_denorm
    
    def __compute_translation(self, idxs, H, size =  [5000,10000]):
    
        min_x = 0
        min_y = 0
        max_x = 0
        max_y = 0

        #For each image
        for i in idxs:

             ## translate
            (Height, Width, _) = self.imgs[i].shape

            # Taking the matrix of initial coordinates of the corners of the secondary image
            # Stored in the following format: [[x1, x2, x3, x4], [y1, y2, y3, y4], [1, 1, 1, 1]]
            # Where (xt, yt) is the coordinate of the i th corner of the image. 
            InitialMatrix = np.array([[0, Width - 1, Width - 1, 0],
                                      [0, 0, Height - 1, Height - 1],
                                      [1, 1, 1, 1]])

            # Finding the final coordinates of the corners of the image after transformation.
            # NOTE: Here, the coordinates of the corners of the frame may go out of the 
            # frame(negative values). We will correct this afterwards by updating the 
            # homography matrix accordingly.
            FinalMatrix = np.dot(H[i], InitialMatrix)

            [x, y, c] = FinalMatrix
            x = np.divide(x, c)
            y = np.divide(y, c)

            c_min_x = min(x)
            c_min_y = min(y)
            c_max_x = max(x)
            c_max_y = max(y)

            if c_min_x < min_x:
                min_x = c_min_x

            if c_min_y < min_y:
                min_y = c_min_y

            if c_max_x > max_x:
                max_x = c_max_x

            if c_max_y > max_y:
                max_y = c_max_y

       
        t = [-min_x,-min_y]
        dy = int(min(max_y - min_y, size[0]))
        dx = int(min(max_x - min_x, size[1]))
        size = [dy,dx]
        

        Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
        return Ht, size
    
    def stitch_images(self, H, idx_ref, T_norm, idxs=None, beautify=True, size = [5000,10000]):
        
        num = len(self.imgs)
        if not idxs:
            idxs = list(range(num))
        
        H_denorm = self.__denormalize_homographies(H, T_norm)
        
        if beautify:
            Ht, size = self.__compute_translation(idxs, H_denorm, size)
        else:
            Ht = np.eye(3)
        
        H_norm_transl = [( T_norm @ Ht @ np.linalg.inv(T_norm) @ h) for h in H]
        stitch = np.zeros(size + [3], dtype=np.uint8)
       
        #For each image
        for i in idxs:
            #Apply the homography        
            img_proj = cv.warpPerspective(self.imgs[i], Ht @ H_denorm[i], size[::-1])

            #Use maximum as stitch operator (very simple stitching mechanism)
            stitch = np.maximum(stitch,img_proj)


        return H_norm_transl, stitch
    
    
   
    
    def stitch_images_denorm(self, H, idx_ref, idxs=None, beautify=True, size = [5000,10000]):
        
        num = len(self.imgs)
        if not idxs:
            idxs = list(range(num))
        
        if beautify:
            Ht, size = self.__compute_translation_2(idxs, H, size)
        else:
            Ht = np.eye(3)
        
        stitch = np.zeros(size + [3], dtype=np.uint8)
       
        #For each image
        for i in idxs:

            #Apply the homography        

            img_proj = cv.warpPerspective(self.imgs[i], Ht @  H[i] , size[::-1])


            #Use maximum as stitch operator (very simple stitching mechanism)
            stitch = np.maximum(stitch,img_proj)

        return Ht, stitch